1. Add more data
2. Exam the feature importance
3. Gric Search
4. Cross validation

In [42]:
import pandas as pd
import numpy as np
import sklearn as sk
import os
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

import joblib

In [44]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

pd.options.display.max_rows = 4000

In [3]:
df_t = pd.read_csv('./train/2024-11/de/trackers.csv')

In [4]:
df_t.reach.describe()

count    711.000000
mean       0.006283
std        0.023132
min        0.000010
25%        0.000266
50%        0.000861
75%        0.003431
max        0.340902
Name: reach, dtype: float64

In [9]:
# import data
# tracker
# site

def read_all_csv_files(root_folder):
    df_tracker = pd.DataFrame()
    df_site = pd.DataFrame()
    popular_tracker = []
    for dirpath,dirnames,filenames in os.walk(root_folder):
        temp = dirpath.split('\\')
        country = temp[-1]
        for filename in filenames:
            if filename == 'sites.csv':
                path = os.path.join(dirpath,filename)
                df = pd.read_csv(path)
                df['country']=country

                df_site = pd.concat([df_site,df])
                
            if filename == 'sites_trackers.csv':
                path = os.path.join(dirpath,filename)
                df = pd.read_csv(path)
                df['country']=country

                df_tracker = pd.concat([df_tracker,df])
            
            if filename == 'trackers.csv':
                path = os.path.join(dirpath,filename)
                df = pd.read_csv(path)
                df_popular_tracker = df[df['reach']>0.01]['tracker']
                popular_tracker=popular_tracker+ (list(df_popular_tracker))
                popular_tracker = list(set(popular_tracker))

                

    return df_tracker,df_site,popular_tracker





In [10]:
df_tracker, df_site, popular_tracker = read_all_csv_files('C:/Users/ywang/Desktop/CSE6242/project/assets')

In [11]:
len(popular_tracker)

161

In [5]:
df_tracker.head()

,Unnamed: 0,month,country,site,tracker,site_proportion,tracker_proportion,site_rank,tracker_rank,cookies,bad_qs,tracked,https,requests,requests_tracking,content_length,requests_failed,has_blocking,script,iframe,beacon,image,stylesheet,font,xhr,plugin,media,referer_leaked,referer_leaked_header,referer_leaked_url,cookie_samesite_none,t_active,hosts,trackers,companies
0,0,2023-01,de,01net.com,google_tag_manager,0.972618,0.000014,1,2012,0.000000,0.000000,0.000000,1.0,1.725225,0.000000,73770.325450,0.599099,0.050676,1.000000,0.0,0.000000,0.006757,0.0,0.0,0.000000,0.0,0.0,0.293919,0.293919,0.006757,0.0,53488.280405,1.00000,1.0,1.0
1,1,2023-01,de,01net.com,google_analytics,0.406353,0.000009,2,2489,0.000000,0.008086,0.008086,1.0,2.520216,0.016173,975.126685,2.353100,0.970350,0.997305,0.0,0.787062,0.043127,0.0,0.0,0.010782,0.0,0.0,0.053908,0.000000,0.053908,0.0,33678.539084,1.00000,1.0,1.0
2,2,2023-01,de,01net.com,google,0.228916,0.000005,3,2638,0.033493,0.000000,0.033493,1.0,2.602871,0.038278,342.875598,2.511962,0.952153,0.023923,0.0,0.971292,0.669856,0.0,0.0,0.052632,0.0,0.0,0.009569,0.000000,0.009569,0.0,55349.727273,1.76555,1.0,1.0
3,3,2023-01,de,01net.com,google_syndication,0.211391,0.000016,4,1092,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,0.000000,1.000000,0.963731,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,20172.886010,1.00000,1.0,1.0
4,4,2023-01,de,01net.com,sirdata,0.210296,0.005678,5,18,0.000000,0.000000,0.000000,1.0,1.052083,0.000000,0.000000,1.052083,0.963542,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,18240.531250,1.00000,1.0,1.0


In [6]:
df_site.head()

,Unnamed: 0,month,country,site,category,popularity,cookies,bad_qs,tracked,https,requests,requests_tracking,content_length,requests_failed,has_blocking,script,iframe,beacon,image,stylesheet,font,xhr,plugin,media,referer_leaked,referer_leaked_header,referer_leaked_url,cookie_samesite_none,t_active,hosts,trackers,companies
0,0,2023-01,de,google.com,Reference,1.000000,0.215257,0.001438,0.215743,0.996181,74.347519,3.023654,2.184856e+06,10.302041,0.224686,0.973823,0.174298,0.038145,0.908518,0.676581,0.793381,0.280732,0.000033,0.055705,0.134243,0.074429,0.038321,0.090361,3.427659e+05,2.943834,2.576978,1.281098
1,1,2023-01,de,amazon.de,E-Commerce,0.671680,0.183577,0.000322,0.183686,0.999167,173.320637,0.720510,2.133588e+07,13.524550,0.754830,0.998019,0.901944,0.022414,0.731136,0.764891,0.552986,0.699111,0.000188,0.088171,0.287516,0.285781,0.001411,0.177631,1.595715e+05,4.880770,3.538968,1.792779
2,2,2023-01,de,youtube.com,Entertainment,0.588916,0.617352,0.000799,0.617523,0.999731,278.700657,1.594176,1.123247e+08,63.582329,0.833384,0.969372,0.211212,0.070232,0.984579,0.820181,0.851004,0.997134,0.000140,0.001312,0.481025,0.334878,0.057039,0.046106,1.308633e+06,8.048141,6.918630,1.232567
3,3,2023-01,de,ebay-kleinanzeigen.de,E-Commerce,0.367534,0.168637,0.028536,0.173068,0.999940,22.289614,5.166571,1.492515e+05,8.779718,0.888224,0.999283,0.122275,0.039566,0.839962,0.063655,0.047233,0.356109,0.000162,0.000343,0.278295,0.036593,0.183224,0.082707,9.406735e+04,8.668500,8.312421,5.945480
4,4,2023-01,de,ebay.de,E-Commerce,0.267930,0.648198,0.004949,0.648309,0.649189,72.551360,8.190588,1.932319e+06,10.422731,0.871691,0.998753,0.485518,0.035776,0.960250,0.637839,0.442374,0.481589,0.000300,0.001986,0.189394,0.156611,0.030003,0.054040,1.190448e+05,4.152513,2.424006,2.090171


In [12]:
# get site_tracker

df_tracker = df_tracker[df_tracker.tracker.isin(popular_tracker)].copy()

df_tracker_site=pd.get_dummies(df_tracker,prefix = None,columns = ['tracker'])

df_tracker_site

,Unnamed: 0,month,country,site,site_proportion,tracker_proportion,site_rank,tracker_rank,cookies,bad_qs,tracked,https,requests,requests_tracking,content_length,requests_failed,has_blocking,script,iframe,beacon,image,stylesheet,font,xhr,plugin,media,referer_leaked,referer_leaked_header,referer_leaked_url,cookie_samesite_none,t_active,hosts,trackers,companies,tracker_1plusx,tracker_1und1,tracker_aaxads.com,tracker_ab_tasty,tracker_ad_lightning,tracker_addthis,tracker_adform,tracker_adition,tracker_adobe_audience_manager,tracker_adobe_dynamic_tag_management,tracker_adobe_experience_cloud,tracker_adobe_typekit,tracker_adscale,tracker_aggregate_knowledge,tracker_akamai_technologies,tracker_amazon_adsystem,tracker_amazon_cdn,tracker_amazon_cloudfront,tracker_amazon_web_services,tracker_amplitude,tracker_appnexus,tracker_at_internet,tracker_azureedge.net,tracker_bidswitch,tracker_bing_ads,tracker_bluekai,tracker_bootstrap,tracker_branch_metrics,tracker_bugsnag,tracker_chartbeat,tracker_clever_push,tracker_cloudflare,tracker_contentsquare.net,tracker_cookiebot,tracker_crazy_egg,tracker_criteo,tracker_dailymotion,tracker_datadome,tracker_didomi,tracker_disqus,tracker_doubleclick,tracker_doubleverify,tracker_ebay,tracker_exoclick,tracker_facebook,tracker_facebook_cdn,tracker_fontawesome_com,tracker_gemius,tracker_google,tracker_google_adservices,tracker_google_analytics,tracker_google_fonts,tracker_google_photos,tracker_google_syndication,tracker_google_tag_manager,tracker_google_users,tracker_googleapis.com,tracker_gravatar,tracker_gstatic,tracker_heap,tracker_hotjar,tracker_hubvisor.io,tracker_iab_consent,tracker_improve_digital,tracker_index_exchange,tracker_infonline,tracker_integral_ad_science,tracker_ispot.tv,tracker_jetpack,tracker_jquery,tracker_jsdelivr,tracker_jw_player,tracker_kameleoon,tracker_kampyle,tracker_kaspersky-labs.com,tracker_krux_digital,tracker_le_monde.fr,tracker_linkedin_analytics,tracker_live_intent,tracker_liveinternet,tracker_liveperson,tracker_liveramp,tracker_lotame,tracker_mail.ru_group,tracker_mbr_targeting,tracker_media.net,tracker_microsoft,tracker_microsoft_clarity,tracker_moat,tracker_nativo,tracker_netratings_sitecensus,tracker_new_relic,tracker_onesignal,tracker_onetag,tracker_onetrust,tracker_openx,tracker_optanaon,tracker_optimizely,tracker_orange,tracker_otto.de,tracker_outbrain,tracker_parsely,tracker_paypal,tracker_pinterest,tracker_polyfill.io,tracker_pubmatic,tracker_qualtrics,tracker_quantcast,tracker_reddit,tracker_rtb_house,tracker_rtl_group,tracker_rubicon,tracker_sailthru_horizon,tracker_schneevonmorgen.com,tracker_scorecard_research_beacon,tracker_seeding_alliance,tracker_segment,tracker_sentry,tracker_shopify_stats,tracker_smart_adserver,tracker_snapchat,tracker_soasta_mpulse,tracker_sovrn,tracker_speedcurve,tracker_spot.im,tracker_spotxchange,tracker_stripe.com,tracker_stroer_digital_media,tracker_taboola,tracker_tag_commander,tracker_teads,tracker_tealium,tracker_the_adex,tracker_tiktok_analytics,tracker_tradedesk,tracker_traffic_stars,tracker_tremor_video,tracker_triplelift,tracker_trusted_shops,tracker_twitter,tracker_twitter_ads,tracker_twitter_syndication,tracker_united_internet_media_gmbh,tracker_unpkg.com,tracker_usabilla,tracker_userreport,tracker_vg_wort,tracker_videoplayerhub.com,tracker_vkontakte,tracker_wikimedia.org,tracker_woopic.com,tracker_wordpress_stats,tracker_xhamster,tracker_yahoo,tracker_yahoo_analytics,tracker_yahoo_japan_retargeting,tracker_yandex,tracker_yandex_metrika,tracker_yieldlab,tracker_yieldlove,tracker_youtube
0,0,2023-01,de,01net.com,0.972618,0.000014,1,2012,0.000000,0.000000,0.000000,1.0,1.725225,0.000000,73770.325450,0.599099,0.050676,1.000000,0.0,0.000000,0.006757,0.0,0.0,0.000000,0.0,0.0,0.293919,0.293919,0.006757,0.0,53488.280405,1.00000,1.0,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [13]:
df_tracker_site=df_tracker_site[['site','month']+['tracker_'+x for x in list(df_tracker.tracker.drop_duplicates())]].copy().drop_duplicates()

In [14]:
df_tracker_site.head() 

,site,month,tracker_google_tag_manager,tracker_google_analytics,tracker_google,tracker_google_syndication,tracker_sentry,tracker_google_fonts,tracker_gravatar,tracker_doubleclick,tracker_twitter,tracker_youtube,tracker_twitter_syndication,tracker_googleapis.com,tracker_kaspersky-labs.com,tracker_dailymotion,tracker_gstatic,tracker_jetpack,tracker_wordpress_stats,tracker_pubmatic,tracker_fontawesome_com,tracker_criteo,tracker_lotame,tracker_index_exchange,tracker_appnexus,tracker_rubicon,tracker_sovrn,tracker_tradedesk,tracker_liveramp,tracker_facebook_cdn,tracker_doubleverify,tracker_jsdelivr,tracker_onetrust,tracker_google_adservices,tracker_amazon_web_services,tracker_amazon_adsystem,tracker_triplelift,tracker_smart_adserver,tracker_yahoo,tracker_improve_digital,tracker_openx,tracker_bidswitch,tracker_adform,tracker_yahoo_analytics,tracker_rtb_house,tracker_quantcast,tracker_adobe_audience_manager,tracker_cloudflare,tracker_bluekai,tracker_adition,tracker_facebook,tracker_amazon_cloudfront,tracker_integral_ad_science,tracker_amazon_cdn,tracker_moat,tracker_taboola,tracker_twitter_ads,tracker_onesignal,tracker_bootstrap,tracker_1plusx,tracker_new_relic,tracker_akamai_technologies,tracker_yieldlab,tracker_adobe_typekit,tracker_rtl_group,tracker_clever_push,tracker_otto.de,tracker_outbrain,tracker_vg_wort,tracker_google_photos,tracker_infonline,tracker_reddit,tracker_seeding_alliance,tracker_trusted_shops,tracker_addthis,tracker_jw_player,tracker_tiktok_analytics,tracker_gemius,tracker_chartbeat,tracker_yandex_metrika,tracker_paypal,tracker_google_users,tracker_microsoft,tracker_disqus,tracker_stripe.com,tracker_media.net,tracker_teads,tracker_jquery,tracker_liveinternet,tracker_iab_consent,tracker_bugsnag,tracker_yandex,tracker_vkontakte,tracker_mail.ru_group,tracker_united_internet_media_gmbh,tracker_1und1,tracker_tealium,tracker_bing_ads,tracker_optimizely,tracker_schneevonmorgen.com,tracker_optanaon,tracker_scorecard_research_beacon,tracker_didomi,tracker_hubvisor.io,tracker_unpkg.com,tracker_tremor_video,tracker_heap,tracker_hotjar,tracker_qualtrics,tracker_live_intent,tracker_netratings_sitecensus,tracker_yieldlove,tracker_adscale,tracker_microsoft_clarity,tracker_spotxchange,tracker_pinterest,tracker_krux_digital,tracker_speedcurve,tracker_exoclick,tracker_traffic_stars,tracker_at_internet,tracker_mbr_targeting,tracker_aggregate_knowledge,tracker_adobe_dynamic_tag_management,tracker_adobe_experience_cloud,tracker_linkedin_analytics,tracker_usabilla,tracker_polyfill.io,tracker_amplitude,tracker_parsely,tracker_videoplayerhub.com,tracker_spot.im,tracker_the_adex,tracker_aaxads.com,tracker_userreport,tracker_soasta_mpulse,tracker_kampyle,tracker_stroer_digital_media,tracker_ispot.tv,tracker_ad_lightning,tracker_cookiebot,tracker_branch_metrics,tracker_snapchat,tracker_wikimedia.org,tracker_azureedge.net,tracker_crazy_egg,tracker_liveperson,tracker_yahoo_japan_retargeting,tracker_nativo,tracker_tag_commander,tracker_contentsquare.net,tracker_onetag,tracker_datadome,tracker_segment,tracker_kameleoon,tracker_xhamster,tracker_sailthru_horizon,tracker_shopify_stats,tracker_ebay,tracker_ab_tasty,tracker_le_monde.fr,tracker_orange,tracker_woopic.com
0,01net.com,2023-01,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [15]:
df_tracker_site = df_tracker_site.groupby(['site','month']).sum()

In [16]:
df_tracker_site.columns = [x.replace('tracker_','') for x in df_tracker_site.columns]

In [17]:
df_tracker_site

google_tag_manager  google_analytics  google  \
site              month                                                   
000webhostapp.com 2023-01                   0                 0       0   
                  2023-02                   0                 0       0   
0123movie.net     2023-03                   1                 1       1   
0123movie.onl     2023-03                   1                 1       1   
0123movie.ru      2023-03                   1                 1       1   
...                                       ...               ...     ...   
zyngagames.com    2023-01                   1                 1       1   
                  2023-02                   1                 1       1   
                  2023-03                   1                 1       1   
zzounds.com       2023-03                   1                 1       1   
zzztube.com       2023-03                   0                 0       0   

                           google_syndication  sentry  google_fonts  gravatar  \
site              month                                                         
000webhostapp.com 2023-01                   0       0             0         0   
                  2023-02                   0       0             1         1   
0123movie.net     2023-03                   0       0             1         0   
0123movie.onl     2023-03                   0       0             1         0   
0123movie.ru      2023-03                   0       0             1         0   
...                                       ...     ...           ...       ...   
zyngagames.com    2023-01                   1       0             1         0   
                  2023-02                   1       0             1         0   
                  2023-03                   0       0             1         0   
zzounds.com       2023-03                   0       1             1         0   
zzztube.com       2023-03                   0       0             0         0   

                           doubleclick  twitter  youtube  twitter_syndication  \
site              month                                                         
000webhostapp.com 2023-01            0        0        0                    0   
                  2023-02            0        0        0                    0   
0123movie.net     2023-03            0        0        0                    0   
0123movie.onl     2023-03            0        0        0                    0   
0123movie.ru      2023-03            0        0        0                    0   
...                                ...      ...      ...                  ...   
zyngagames.com    2023-01            1        1        1                    1   
                  2023-02            1        1        1                    1   
                  2023-03            1        1        1                    1   
zzounds.com       2023-03            0        0        1                    0   
zzztube.com       2023-03            0        0        0                    0   

                           googleapis.com  kaspersky-labs.com  dailymotion  \
site              month                                                      
000webhostapp.com 2023-01               0                   0            0   
                  2023-02               1                   0            0   
0123movie.net     2023-03               0                   0            0   
0123movie.onl     2023-03               1                   0            0   
0123movie.ru      2023-03               1                   0            0   
...                                   ...                 ...          ...   
zyngagames.com    2023-01               1                   1            0   
                  2023-02               1                   1            0   
                  2023-03               1                   1            0   
zzounds.com       2023-03               1                   0            0   
zzztube

In [18]:
df_site['de'] = df_site.country.apply(lambda x: 1 if x == 'de' else 0)
df_site['eu'] = df_site.country.apply(lambda x: 1 if x == 'eu' else 0)
df_site['fr'] = df_site.country.apply(lambda x: 1 if x == 'fr' else 0)
df_site['global'] = df_site.country.apply(lambda x: 1 if x == 'global' else 0)
df_site['us'] = df_site.country.apply(lambda x: 1 if x == 'us' else 0)

In [19]:
df_site

,Unnamed: 0,month,country,site,category,popularity,cookies,bad_qs,tracked,https,requests,requests_tracking,content_length,requests_failed,has_blocking,script,iframe,beacon,image,stylesheet,font,xhr,plugin,media,referer_leaked,referer_leaked_header,referer_leaked_url,cookie_samesite_none,t_active,hosts,trackers,companies,de,eu,fr,global,us
0,0,2023-01,de,google.com,Reference,1.000000,0.215257,0.001438,0.215743,0.996181,74.347519,3.023654,2.184856e+06,10.302041,0.224686,0.973823,0.174298,0.038145,0.908518,0.676581,0.793381,0.280732,0.000033,0.055705,0.134243,0.074429,0.038321,0.090361,3.427659e+05,2.943834,2.576978,1.281098,1,0,0,0,0
1,1,2023-01,de,amazon.de,E-Commerce,0.671680,0.183577,0.000322,0.183686,0.999167,173.320637,0.720510,2.133588e+07,13.524550,0.754830,0.998019,0.901944,0.022414,0.731136,0.764891,0.552986,0.699111,0.000188,0.088171,0.287516,0.285781,0.001411,0.177631,1.595715e+05,4.880770,3.538968,1.792779,1,0,0,0,0
2,2,2023-01,de,youtube.com,Entertainment,0.588916,0.617352,0.000799,0.617523,0.999731,278.700657,1.594176,1.123247e+08,63.582329,0.833384,0.969372,0.211212,0.070232,0.984579,0.820181,0.851004,0.997134,0.000140,0.001312,0.481025,0.334878,0.057039,0.046106,1.308633e+06,8.048141,6.918630,1.232567,1,0,0,0,0
3,3,2023-01,de,ebay-kleinanzeigen.de,E-Commerce,0.367534,0.168637,0.028536,0.173068,0.999940,22.289614,5.166571,1.492515e+05,8.779718,0.888224,0.999283,0.122275,0.039566,0.839962,0.063655,0.047233,0.356109,0.000162,0.000343,0.278295,0.036593,0.183224,0.082707,9.406735e+04,8.668500,8.312421,5.945480,1,0,0,0,0
4,4,2023-01,de,ebay.de,E-Commerce,0.267930,0.648198,0.004949,0.648309,0.649189,72.551360,8.190588,1.932319e+06,10.422731,0.871691,0.998753,0.485518,0.035776,0.960250,0.637839,0.442374,0.481589,0.000300,0.001986,0.189394,0.156611,0.030003,0.054040,1.190448e+05,4.152513,2.424006,2.090171,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,2023-03,us,roughcountry.com,Uncategorized,0.000055,0.790560,0.125000,0.790929,1.000000,92.436209,22.581121,9.403266e+05,29.702065,0.879425,0.998894,0.347345,0.768068,0.972714,0.809366,0.401917,0.975295,0.000000,0.000369,0.681416,0.057153,0.650811,0.688053,2.209913e+05,23.185472,16.618363,12.373156,0,0,0,0,1
9996,9996,2023-03,us,aopa.org,Uncategorized,0.000055,0.689897,0.062684,0.697271,0.997050,315.990413,5.446165,5.903817e+06,30.798673,0.878687,0.992257,0.264012,0.471239,0.621681,0.321903,0.311947,0.934735,0.000000,0.561947,0.550516,0.417404,0.160767,0.654130,4.892768e+05,13.925885,12.402286,7.896386,0,0,0,0,1
9997,9997,2023-03,us,dood.re,Uncategorized,0.000055,0.073009,0.005900,0.073746,1.000000,33.613938,0.498894,1.012547e+06,7.676991,0.757743,0.991519,0.035767,0.013643,0.971608,0.859882,0.865782,0.846239,0.000000,0.002581,0.015487,0.002950,0.008481,0.015118,2.991936e+05,7.000369,3.575959,3.231563,0,0,0,0,1
9998,9998,2023-03,us,myguestaccount.com,Uncategorized,0.000055,0.273700,0.039100,0.286241,0.991516,28.191811,2.054592,6.420797e+05,5.843969,0.638141,0.884913,0.315382,0.167466,0.732571,0.590188,0.545924,0.588344,0.000000,0.000369,0.210623,0.000738,0.188122,0.125415,3.054176e+05,6.111398,5.222796,3.445592,0,0,0,0,1


In [20]:
df_site = df_site.drop(['Unnamed: 0','country'],axis=1)

In [21]:
df_site = pd.get_dummies(df_site,columns=['category']).set_index(['site','month'])

In [22]:
df_site

,,popularity,cookies,bad_qs,tracked,https,requests,requests_tracking,content_length,requests_failed,has_blocking,script,iframe,beacon,image,stylesheet,font,xhr,plugin,media,referer_leaked,referer_leaked_header,referer_leaked_url,cookie_samesite_none,t_active,hosts,trackers,companies,de,eu,fr,global,us,category_Adult,category_Adult,category_Banking,category_Business,category_E-Commerce,category_E-Commerce,category_E-commerce,category_Entertainment,category_Entertainment,category_Government,category_Health,category_News and Portals,category_Political,category_Recreation,category_Reference,category_Uncategorized
site,month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
google.com,2023-01,1.000000,0.215257,0.001438,0.215743,0.996181,74.347519,3.023654,2.184856e+06,10.302041,0.224686,0.973823,0.174298,0.038145,0.908518,0.676581,0.793381,0.280732,0.000033,0.055705,0.134243,0.074429,0.038321,0.090361,3.427659e+05,2.943834,2.576978,1.281098,1,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
amazon.de,2023-01,0.671680,0.183577,0.000322,0.183686,0.999167,173.320637,0.720510,2.133588e+07,13.524550,0.754830,0.998019,0.901944,0.022414,0.731136,0.764891,0.552986,0.699111,0.000188,0.088171,0.287516,0.285781,0.001411,0.177631,1.595715e+05,4.880770,3.538968,1.792779,1,0,0,0,0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
youtube.com,2023-01,0.588916,0.617352,0.000799,0.617523,0.999731,278.700657,1.594176,1.123247e+08,63.582329,0.833384,0.969372,0.211212,0.070232,0.984579,0.820181,0.851004,0.997134,0.000140,0.001312,0.481025,0.334878,0.057039,0.046106,1.308633e+06,8.048141,6.918630,1.232567,1,0,0,0,0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
ebay-kleinanzeigen.de,2023-01,0.367534,0.168637,0.028536,0.173068,0.999940,22.289614,5.166571,1.492515e+05,8.779718,0.888224,0.999283,0.122275,0.039566,0.839962,0.063655,0.047233,0.356109,0.000162,0.000343,0.278295,0.036593,0.183224,0.082707,9.406735e+04,8.668500,8.312421,5.945480,1,0,0,0,0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
ebay.de,2023-01,0.267930,0.648198,0.004949,0.648309,0.649189,72.551360,8.190588,1.932319e+06,10.422731,0.871691,0.998753,0.485518,0.035776,0.960250,0.637839,0.442374,0.481589,0.000300,0.001986,0.189394,0.156611,0.030003,0.054040,1.190448e+05,4.152513,2.424006,2.090171,1,0,0,0,0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
roughcountry.com,2023-03,0.000055,0.790560,0.125000,0.790929,1.000000,92.436209,22.581121,9.403266e+05,29.702065,0.879425,0.998894,0.347345,0.768068,0.972714,0.809366,0.401917,0.975295,0.000000,0.000369,0.681416,0.057153,0.650811,0.688053,2.209913e+05,23.185472,16.618363,12.373156,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
aopa.org,2023-03,0.000055,0.689897,0.062684,0.697271,0.997050,315.990413,5.446165,5.903817e+06,30.798673,0.878687,0.992257,0.264012,0.471239,0.621681,0.321903,0.311947,0.934735,0.000000,0.561947,0.550516,0.417404,0.160767,0.654130,4.892768e+05,13.925885,12.402286,7.896386,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
dood.re,2023-03,0.000055,0.073009,0.005900,0.073746,1.000000,33.613938,0.498894,1.012547e+06,7.676991,0.757743,0.991519,0.035767,0.013643,0.971608,0.859882,0.865782,0.846239,0.000000,0.002581,0.015487,0.002950,0.008481,0.015118,2.991936e+05,7.000369,3.575959,3.231563,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [23]:
df_model = df_site.join(df_tracker_site,how='left')

df_model.head()

,,popularity,cookies,bad_qs,tracked,https,requests,requests_tracking,content_length,requests_failed,has_blocking,script,iframe,beacon,image,stylesheet,font,xhr,plugin,media,referer_leaked,referer_leaked_header,referer_leaked_url,cookie_samesite_none,t_active,hosts,trackers,companies,de,eu,fr,global,us,category_Adult,category_Adult,category_Banking,category_Business,category_E-Commerce,category_E-Commerce,category_E-commerce,category_Entertainment,category_Entertainment,category_Government,category_Health,category_News and Portals,category_Political,category_Recreation,category_Reference,category_Uncategorized,google_tag_manager,google_analytics,google,google_syndication,sentry,google_fonts,gravatar,doubleclick,twitter,youtube,twitter_syndication,googleapis.com,kaspersky-labs.com,dailymotion,gstatic,jetpack,wordpress_stats,pubmatic,fontawesome_com,criteo,lotame,index_exchange,appnexus,rubicon,sovrn,tradedesk,liveramp,facebook_cdn,doubleverify,jsdelivr,onetrust,google_adservices,amazon_web_services,amazon_adsystem,triplelift,smart_adserver,yahoo,improve_digital,openx,bidswitch,adform,yahoo_analytics,rtb_house,quantcast,adobe_audience_manager,cloudflare,bluekai,adition,facebook,amazon_cloudfront,integral_ad_science,amazon_cdn,moat,taboola,twitter_ads,onesignal,bootstrap,1plusx,new_relic,akamai_technologies,yieldlab,adobe_typekit,rtl_group,clever_push,otto.de,outbrain,vg_wort,google_photos,infonline,reddit,seeding_alliance,trusted_shops,addthis,jw_player,tiktok_analytics,gemius,chartbeat,yandex_metrika,paypal,google_users,microsoft,disqus,stripe.com,media.net,teads,jquery,liveinternet,iab_consent,bugsnag,yandex,vkontakte,mail.ru_group,united_internet_media_gmbh,1und1,tealium,bing_ads,optimizely,schneevonmorgen.com,optanaon,scorecard_research_beacon,didomi,hubvisor.io,unpkg.com,tremor_video,heap,hotjar,qualtrics,live_intent,netratings_sitecensus,yieldlove,adscale,microsoft_clarity,spotxchange,pinterest,krux_digital,speedcurve,exoclick,traffic_stars,at_internet,mbr_targeting,aggregate_knowledge,adobe_dynamic_tag_management,adobe_experience_cloud,linkedin_analytics,usabilla,polyfill.io,amplitude,parsely,videoplayerhub.com,spot.im,the_adex,aaxads.com,userreport,soasta_mpulse,kampyle,stroer_digital_media,ispot.tv,ad_lightning,cookiebot,branch_metrics,snapchat,wikimedia.org,azureedge.net,crazy_egg,liveperson,yahoo_japan_retargeting,nativo,tag_commander,contentsquare.net,onetag,datadome,segment,kameleoon,xhamster,sailthru_horizon,shopify_stats,ebay,ab_tasty,le_monde.fr,orange,woopic.com
site,month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
google.com,2023-01,1.000000,0.215257,0.001438,0.215743,0.996181,74.347519,3.023654,2.184856e+06,10.302041,0.224686,0.973823,0.174298,0.038145,0.908518,0.676581,0.793381,0.280732,0.000033,0.055705,0.134243,0.074429,0.038321,0.090361,3.427659e+05,2.943834,2.576978,1.281098,1,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
amazon.de,2023-01,0.671680,0.183577,0.000322,0.183686,0.999167,173.320637,0.720510,2.133588e+07,13.524550,0.754830,0.998019,0.901944,0.022414,0.731136,0.764891,0.552986,0.699111,0.000188,0.088171,0.287516,0.285781,0.001411,0.177631,1.5957

In [24]:
check = df_model.isnull().sum()

check[check>0]

google_tag_manager              246
google_analytics                246
google                          246
google_syndication              246
sentry                          246
google_fonts                    246
gravatar                        246
doubleclick                     246
twitter                         246
youtube                         246
twitter_syndication             246
googleapis.com                  246
kaspersky-labs.com              246
dailymotion                     246
gstatic                         246
jetpack                         246
wordpress_stats                 246
pubmatic                        246
fontawesome_com                 246
criteo                          246
lotame                          246
index_exchange                  246
appnexus                        246
rubicon                         246
sovrn                           246
tradedesk                       246
liveramp                        246
facebook_cdn                

In [25]:
df_model.shape

(106000, 209)

In [26]:
df_site.shape

(106000, 48)

In [27]:
trackers = list(df_tracker.tracker.drop_duplicates())
X = df_model.drop(trackers,axis=1)
X_columns = X.columns
y = df_model[trackers]

In [28]:
del df_site
del df_model
del df_tracker
del df_tracker_site

In [47]:
trackers

['google_tag_manager',
 'google_analytics',
 'google',
 'google_syndication',
 'sentry',
 'google_fonts',
 'gravatar',
 'doubleclick',
 'twitter',
 'youtube',
 'twitter_syndication',
 'googleapis.com',
 'kaspersky-labs.com',
 'dailymotion',
 'gstatic',
 'jetpack',
 'wordpress_stats',
 'pubmatic',
 'fontawesome_com',
 'criteo',
 'lotame',
 'index_exchange',
 'appnexus',
 'rubicon',
 'sovrn',
 'tradedesk',
 'liveramp',
 'facebook_cdn',
 'doubleverify',
 'jsdelivr',
 'onetrust',
 'google_adservices',
 'amazon_web_services',
 'amazon_adsystem',
 'triplelift',
 'smart_adserver',
 'yahoo',
 'improve_digital',
 'openx',
 'bidswitch',
 'adform',
 'yahoo_analytics',
 'rtb_house',
 'quantcast',
 'adobe_audience_manager',
 'cloudflare',
 'bluekai',
 'adition',
 'facebook',
 'amazon_cloudfront',
 'integral_ad_science',
 'amazon_cdn',
 'moat',
 'taboola',
 'twitter_ads',
 'onesignal',
 'bootstrap',
 '1plusx',
 'new_relic',
 'akamai_technologies',
 'yieldlab',
 'adobe_typekit',
 'rtl_group',
 'cleve

In [30]:
X

,,popularity,cookies,bad_qs,tracked,https,requests,requests_tracking,content_length,requests_failed,has_blocking,script,iframe,beacon,image,stylesheet,font,xhr,plugin,media,referer_leaked,referer_leaked_header,referer_leaked_url,cookie_samesite_none,t_active,hosts,trackers,companies,de,eu,fr,global,us,category_Adult,category_Adult,category_Banking,category_Business,category_E-Commerce,category_E-Commerce,category_E-commerce,category_Entertainment,category_Entertainment,category_Government,category_Health,category_News and Portals,category_Political,category_Recreation,category_Reference,category_Uncategorized
site,month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
google.com,2023-01,1.000000,0.215257,0.001438,0.215743,0.996181,74.347519,3.023654,2.184856e+06,10.302041,0.224686,0.973823,0.174298,0.038145,0.908518,0.676581,0.793381,0.280732,0.000033,0.055705,0.134243,0.074429,0.038321,0.090361,3.427659e+05,2.943834,2.576978,1.281098,1,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
amazon.de,2023-01,0.671680,0.183577,0.000322,0.183686,0.999167,173.320637,0.720510,2.133588e+07,13.524550,0.754830,0.998019,0.901944,0.022414,0.731136,0.764891,0.552986,0.699111,0.000188,0.088171,0.287516,0.285781,0.001411,0.177631,1.595715e+05,4.880770,3.538968,1.792779,1,0,0,0,0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
youtube.com,2023-01,0.588916,0.617352,0.000799,0.617523,0.999731,278.700657,1.594176,1.123247e+08,63.582329,0.833384,0.969372,0.211212,0.070232,0.984579,0.820181,0.851004,0.997134,0.000140,0.001312,0.481025,0.334878,0.057039,0.046106,1.308633e+06,8.048141,6.918630,1.232567,1,0,0,0,0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
ebay-kleinanzeigen.de,2023-01,0.367534,0.168637,0.028536,0.173068,0.999940,22.289614,5.166571,1.492515e+05,8.779718,0.888224,0.999283,0.122275,0.039566,0.839962,0.063655,0.047233,0.356109,0.000162,0.000343,0.278295,0.036593,0.183224,0.082707,9.406735e+04,8.668500,8.312421,5.945480,1,0,0,0,0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
ebay.de,2023-01,0.267930,0.648198,0.004949,0.648309,0.649189,72.551360,8.190588,1.932319e+06,10.422731,0.871691,0.998753,0.485518,0.035776,0.960250,0.637839,0.442374,0.481589,0.000300,0.001986,0.189394,0.156611,0.030003,0.054040,1.190448e+05,4.152513,2.424006,2.090171,1,0,0,0,0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
roughcountry.com,2023-03,0.000055,0.790560,0.125000,0.790929,1.000000,92.436209,22.581121,9.403266e+05,29.702065,0.879425,0.998894,0.347345,0.768068,0.972714,0.809366,0.401917,0.975295,0.000000,0.000369,0.681416,0.057153,0.650811,0.688053,2.209913e+05,23.185472,16.618363,12.373156,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
aopa.org,2023-03,0.000055,0.689897,0.062684,0.697271,0.997050,315.990413,5.446165,5.903817e+06,30.798673,0.878687,0.992257,0.264012,0.471239,0.621681,0.321903,0.311947,0.934735,0.000000,0.561947,0.550516,0.417404,0.160767,0.654130,4.892768e+05,13.925885,12.402286,7.896386,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
dood.re,2023-03,0.000055,0.073009,0.005900,0.073746,1.000000,33.613938,0.498894,1.012547e+06,7.676991,0.757743,0.991519,0.035767,0.013643,0.971608,0.859882,0.865782,0.846239,0.000000,0.002581,0.015487,0.002950,0.008481,0.015118,2.991936e+05,7.000369,3.575959,3.231563,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [31]:
X_columns

Index(['popularity', 'cookies', 'bad_qs', 'tracked', 'https', 'requests',
       'requests_tracking', 'content_length', 'requests_failed',
       'has_blocking', 'script', 'iframe', 'beacon', 'image', 'stylesheet',
       'font', 'xhr', 'plugin', 'media', 'referer_leaked',
       'referer_leaked_header', 'referer_leaked_url', 'cookie_samesite_none',
       't_active', 'hosts', 'trackers', 'companies', 'de', 'eu', 'fr',
       'global', 'us', 'category_Adult', 'category_Adult ', 'category_Banking',
       'category_Business', 'category_E-Commerce', 'category_E-Commerce ',
       'category_E-commerce', 'category_Entertainment',
       'category_Entertainment ', 'category_Government', 'category_Health',
       'category_News and Portals', 'category_Political',
       'category_Recreation', 'category_Reference', 'category_Uncategorized'],
      dtype='object')

In [48]:
len(X_columns)

48

In [32]:
y

,,google_tag_manager,google_analytics,google,google_syndication,sentry,google_fonts,gravatar,doubleclick,twitter,youtube,twitter_syndication,googleapis.com,kaspersky-labs.com,dailymotion,gstatic,jetpack,wordpress_stats,pubmatic,fontawesome_com,criteo,lotame,index_exchange,appnexus,rubicon,sovrn,tradedesk,liveramp,facebook_cdn,doubleverify,jsdelivr,onetrust,google_adservices,amazon_web_services,amazon_adsystem,triplelift,smart_adserver,yahoo,improve_digital,openx,bidswitch,adform,yahoo_analytics,rtb_house,quantcast,adobe_audience_manager,cloudflare,bluekai,adition,facebook,amazon_cloudfront,integral_ad_science,amazon_cdn,moat,taboola,twitter_ads,onesignal,bootstrap,1plusx,new_relic,akamai_technologies,yieldlab,adobe_typekit,rtl_group,clever_push,otto.de,outbrain,vg_wort,google_photos,infonline,reddit,seeding_alliance,trusted_shops,addthis,jw_player,tiktok_analytics,gemius,chartbeat,yandex_metrika,paypal,google_users,microsoft,disqus,stripe.com,media.net,teads,jquery,liveinternet,iab_consent,bugsnag,yandex,vkontakte,mail.ru_group,united_internet_media_gmbh,1und1,tealium,bing_ads,optimizely,schneevonmorgen.com,optanaon,scorecard_research_beacon,didomi,hubvisor.io,unpkg.com,tremor_video,heap,hotjar,qualtrics,live_intent,netratings_sitecensus,yieldlove,adscale,microsoft_clarity,spotxchange,pinterest,krux_digital,speedcurve,exoclick,traffic_stars,at_internet,mbr_targeting,aggregate_knowledge,adobe_dynamic_tag_management,adobe_experience_cloud,linkedin_analytics,usabilla,polyfill.io,amplitude,parsely,videoplayerhub.com,spot.im,the_adex,aaxads.com,userreport,soasta_mpulse,kampyle,stroer_digital_media,ispot.tv,ad_lightning,cookiebot,branch_metrics,snapchat,wikimedia.org,azureedge.net,crazy_egg,liveperson,yahoo_japan_retargeting,nativo,tag_commander,contentsquare.net,onetag,datadome,segment,kameleoon,xhamster,sailthru_horizon,shopify_stats,ebay,ab_tasty,le_monde.fr,orange,woopic.com
site,month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
google.com,2023-01,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
amazon.de,2023-01,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
youtube.com,2023-01,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [33]:
X=X.fillna(0).astype('float').to_numpy()
y=y.fillna(0).astype('float').to_numpy()

In [34]:
X

array([[1.00000000e+00, 2.15256681e-01, 1.43843143e-03, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.71680343e-01, 1.83576976e-01, 3.22432064e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.88915999e-01, 6.17352115e-01, 7.98783118e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [5.54085644e-05, 7.30088496e-02, 5.89970501e-03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [5.53881335e-05, 2.73699742e-01, 3.90999631e-02, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [5.53677026e-05, 1.27306273e-01, 7.74907749e-03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [35]:
y

array([[1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
multi_label_rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [38]:
multi_label_rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [39]:
y_pred=multi_label_rf.predict(X_test)

In [41]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96     17547
           1       0.93      0.99      0.96     18067
           2       0.96      0.99      0.98     19291
           3       0.93      0.85      0.89      9410
           4       0.91      0.66      0.77      6700
           5       0.94      1.00      0.97     19464
           6       0.99      0.38      0.55      1945
           7       0.95      0.98      0.96     17073
           8       0.92      0.77      0.84      6705
           9       0.87      0.87      0.87     10007
          10       0.93      0.70      0.80      4698
          11       0.90      0.98      0.94     17058
          12       0.80      0.97      0.88     14842
          13       1.00      0.45      0.62       484
          14       0.96      1.00      0.98     19897
          15       0.98      0.30      0.46       570
          16       0.98      0.34      0.51      1474
          17       0.91    

c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [43]:
joblib.dump(multi_label_rf,'trackers_model.joblib')

['trackers_model.joblib']